# 检索进阶

在基础的 RAG 流程中，依赖向量相似度从知识库中检索信息。

这种方法存在一些固有的局限性，例如最相关的文档不总是在检索结果的顶端，以及语义理解的偏差等。

为了构建更强大、更精准的生产级 RAG 应用，需要引入更高级的检索技术。



## 重排序 (Re-ranking)

### RRF (Reciprocal Rank Fusion)

RRF 是一种简单而有效的零样本重排方法，不依赖于任何模型训练，

纯粹基于文档在多个不同检索器结果列表中的排名来计算最终分数。

### RankLLM / LLM-based Reranker

![](images/4_5_2.webp)

RankLLM **代表了一类**直接利用大型语言模型本身来进行重排的方法，这种方法通过一个精心设计的提示词来实现。

该提示词会包含用户的查询和一系列候选文档（通常是文档的摘要或关键部分），

然后要求 LLM 以特定格式（如 JSON）输出一个排序后的文档列表，并给出每个文档的相关性分数。

In [ ]:
prompt = """

以下是一个文档列表，每个文档都有一个编号和摘要。同时提供一个问题。请根据问题，按相关性顺序列出您认为需要查阅的文档编号，并给出相关性分数（1-10分）。请不要包含与问题无关的文档。

示例格式:
文档 1: <文档1的摘要>
文档 2: <文档2的摘要>
...
文档 10: <文档10的摘要>

问题: <用户的问题>

回答:
Doc: 9, Relevance: 7
Doc: 3, Relevance: 4
Doc: 7, Relevance: 3

"""


### Cross-Encoder 重排

Cross-Encoder（交叉编码器）是将查询（Query）和每个候选文档（Document）拼接成一个单一的输入。

**[CLS] query [SEP] document [SEP]**

将其输入到一个预训练的 Transformer 模型（如 BERT）中，

最终输出一个单一的分数（通常在 0 到 1 之间），这个分数直接代表了文档与查询的相关性。

![](images/4_5_3.svg)

1. 初步检索：搜索引擎首先从知识库中召回一个初始的文档列表（例如，前 50 篇）。

2. 逐一评分：对于列表中的每一篇文档，系统都将其与原始查询配对，然后发送给 Cross-Encoder 模型。

3. 独立推理：模型对每个“查询-文档”对进行一次完整的、独立的推理计算，得出一个精确的相关性分数。

4. 返回重排结果：系统根据这些新的分数对文档列表进行重新排序，并将最终结果返回给用户。

常见的 Cross-Encoder 模型包括 ms-marco-MiniLM-L-12-v2、ms-marco-TinyBERT-L-2-v2 等。

### ColBERT 重排

一种创新的重排模型，采用了一种“后期交互”机制，在 Cross-Encoder 的高精度和双编码器（Bi-Encoder）的高效率之间取得了平衡。

1. 独立编码：ColBERT 分别为查询（Query）和文档（Document）中的每个 Token 生成上下文相关的嵌入向量。
    
    - 这一步是独立完成的，可以预先计算并存储文档的向量，从而加快查询速度。

2. 后期交互：在查询时，模型会计算查询中每个 Token 的向量与文档中每个 Token 向量之间的最大相似度（MaxSim）。

3. 分数聚合：最后，将查询中所有 Token 得到的最大相似度分数相加，得到最终的相关性总分。

### 对比

| 特性 | RRF | RankLLM | Cross-Encoder | ColBERT |
| :--- | :--- | :--- | :--- | :--- |
| **核心机制** | 融合多个排名 | LLM 推理，生成排序列表 | 联合编码查询与文档，计算单一相关分 | 独立编码，后期交互 |
| **计算成本** | 低（简单数学计算） | 中 (API 费用与延迟) | 高（N次模型推理） | 中（向量点积计算） |
| **交互粒度** | 无（仅排名） | 概念/语义级 | 句子级（Query-Doc Pair） | Token 级 |
| **适用场景** | 多路召回结果融合 | 高价值语义理解场景 | Top-K 精排 | Top-K 重排 |


## 压缩 (Compression)

初步检索到的文档块（Chunks）虽然整体上与查询相关，但可能包含大量无关的“噪音”文本。

这些未经处理的、冗长的上下文直接提供给 LLM，不仅会增加 API 调用的成本和延迟，还可能因为信息过载而降低最终生成答案的质量。

压缩的目标就是对检索到的内容进行“压缩”和“提炼”，只保留与用户查询最直接相关的信息。

**实现方式：**

- 内容提取：从文档中只抽出与查询相关的句子或段落。

- 文档过滤：完全丢弃那些虽然被初步召回，但经过更精细判断后认为不相关的整个文档。

### LangChain 的 DocumentCompressor

- LLMChainExtractor: 这是最直接的压缩方式。遍历每个文档，并利用 LLM 来判断并提取出其中与查询相关的部分。

- LLMChainFilter: 同样使用 LLM，判断整个文档是否与查询相关，如果相关，则保留整个文档；如果不相关，则直接丢弃。

- EmbeddingsFilter: 计算查询和每个文档的嵌入向量之间的相似度，只保留那些相似度超过预设阈值的文档。

### 自定义重排器与压缩管道

 LangChain + ColBERT 

In [ ]:
from langchain_core.documents import BaseDocumentCompressor
from typing import Sequence
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from langchain_core.documents import Document

class ColBERTReranker(BaseDocumentCompressor):
    """ColBERT重排器"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        model_name = "models/bert/bert-base-uncased"

        # 加载模型和分词器
        object.__setattr__(self, 'tokenizer', AutoTokenizer.from_pretrained(model_name))
        object.__setattr__(self, 'model', AutoModel.from_pretrained(model_name))
        self.model.eval()
        print(f"ColBERT模型加载完成")

    def encode_text(self, texts):
        """ColBERT文本编码"""
        inputs = self.tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        )

        with torch.no_grad():
            outputs = self.model(**inputs)

        embeddings = outputs.last_hidden_state
        embeddings = F.normalize(embeddings, p=2, dim=-1)

        return embeddings

    def calculate_colbert_similarity(self, query_emb, doc_embs, query_mask, doc_masks):
        """ColBERT相似度计算（MaxSim操作）"""
        scores = []

        for i, doc_emb in enumerate(doc_embs):
            doc_mask = doc_masks[i:i+1]

            # 计算相似度矩阵
            similarity_matrix = torch.matmul(query_emb, doc_emb.unsqueeze(0).transpose(-2, -1))

            # 应用文档mask
            doc_mask_expanded = doc_mask.unsqueeze(1)
            similarity_matrix = similarity_matrix.masked_fill(~doc_mask_expanded.bool(), -1e9)

            # MaxSim操作
            max_sim_per_query_token = similarity_matrix.max(dim=-1)[0]

            # 应用查询mask
            query_mask_expanded = query_mask.unsqueeze(0)
            max_sim_per_query_token = max_sim_per_query_token.masked_fill(~query_mask_expanded.bool(), 0)

            # 求和得到最终分数
            colbert_score = max_sim_per_query_token.sum(dim=-1).item()
            scores.append(colbert_score)

        return scores

    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks=None,
    ) -> Sequence[Document]:
        """对文档进行ColBERT重排序"""
        if len(documents) == 0:
            return documents

        # 编码查询
        query_inputs = self.tokenizer(
            [query],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        )

        with torch.no_grad():
            query_outputs = self.model(**query_inputs)
            query_embeddings = F.normalize(query_outputs.last_hidden_state, p=2, dim=-1)

        # 编码文档
        doc_texts = [doc.page_content for doc in documents]
        doc_inputs = self.tokenizer(
            doc_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        )

        with torch.no_grad():
            doc_outputs = self.model(**doc_inputs)
            doc_embeddings = F.normalize(doc_outputs.last_hidden_state, p=2, dim=-1)

        # 计算ColBERT相似度
        scores = self.calculate_colbert_similarity(
            query_embeddings,
            doc_embeddings,
            query_inputs['attention_mask'],
            doc_inputs['attention_mask']
        )

        # 排序并返回前5个
        scored_docs = list(zip(documents, scores))
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        reranked_docs = [doc for doc, _ in scored_docs[:5]]

        return reranked_docs
    
reranker = ColBERTReranker()


ColBERT模型加载完成


In [ ]:
# 初始化大模型
from langchain_deepseek import ChatDeepSeek
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os
load_dotenv()

hf_bge_embeddings = HuggingFaceEmbeddings(model_name="models/bge/bge-small-zh-v1.5")

llm = ChatDeepSeek(
    model="deepseek-chat", 
    temperature=0.1, 
    api_key=os.getenv("DEEPSEEK_API_KEY")
)


In [ ]:
# 加载和处理文档
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("data/ai.txt", encoding="utf-8")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)


In [ ]:
# 创建向量存储和基础检索器
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, hf_bge_embeddings)
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 20})


In [ ]:
# LLM 压缩器
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)


In [ ]:
# 使用DocumentCompressorPipeline组装压缩管道
# 流程: ColBERT重排 -> LLM压缩
from langchain_classic.retrievers.document_compressors import DocumentCompressorPipeline

pipeline_compressor = DocumentCompressorPipeline(transformers=[reranker, compressor])


In [ ]:
# 创建最终的压缩检索器
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever

final_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor,
    base_retriever=base_retriever
)


In [ ]:
# 执行查询并展示结果
query = "AI还有哪些缺陷需要克服？"
print(f"\n{'='*20} 开始执行查询 {'='*20}")
print(f"查询: {query}\n")

# 基础检索结果
print(f"--- (1) 基础检索结果 (Top 20) ---")
base_results = base_retriever.invoke(query)
for i, doc in enumerate(base_results):
    print(f"  [{i+1}] {doc.page_content[:100]}...\n")

# 使用管道压缩器的最终结果
print(f"\n--- (2) 管道压缩后结果 (ColBERT重排 + LLM压缩) ---")
final_results = final_retriever.invoke(query)
for i, doc in enumerate(final_results):
    print(f"  [{i+1}] {doc.page_content}\n")



==================== 开始执行查询 ====================
查询: AI还有哪些缺陷需要克服？

--- (1) 基础检索结果 (Top 20) ---
  [1] 一个比较明显的问题是，AI生成内容虽然已非常流畅，但提供的信息很多时候还是不准确。5月，日本研究人员在德国《先进科学》杂志发表的一项研究成果中指出，这一问题与人类的语言障碍——失语症类似。

    ...

  [2] 行业巨头谷歌公司也没闲着。该公司在5月推出整体性能和智能推理能力均较以往版本大幅提升的多个“双子座2.5”系列模型，并发布了多个多模态模型，如图像生成模型Imagen 4和视频生成模型Veo 3，具备...

  [3] 业界也确实在努力从不同角度去寻求优化大模型的解决方案。中国科学院自动化研究所联合鹏城实验室提出了一种高效推理策略AutoThink，可让大模型实现自主切换思考模式，避免“过度思考”。

    据研究...

  [4] 一些国家已在积极尝试通过优化政策、法规来营造更好的AI创新环境。日本参议院全体会议5月28日以多数赞成票通过该国首部专门针对AI的法律，旨在促进AI相关技术研发和应用并防止其滥用。依据这部《人工智能相...

  [5] 5月，全球多家科技公司发布新的大模型，它们在语义理解、多模态等方面进一步提升，人工智能（AI）的能力边界在不断扩大。随着无人驾驶、机器人等技术借助AI快速进化并逐步投入市场，不少国家通过推进法规建设、...


--- (2) 管道压缩后结果 (ColBERT重排 + LLM压缩) ---
  [1] 一个比较明显的问题是，AI生成内容虽然已非常流畅，但提供的信息很多时候还是不准确。
大模型在出现严重错误时仍表达流畅，这与感觉性失语症的症状有相似之处，即说话流利却总说不出什么意思。
它们可能被锁定在一种僵化的内部模式中，限制其灵活运用所储存知识。

  [2] 中国科学院自动化研究所联合鹏城实验室提出了一种高效推理策略AutoThink，可让大模型实现自主切换思考模式，避免“过度思考”。

    据研究人员介绍，AutoThink提供了一种简单而有效的推理新范式——通过省略号提示配合三阶段强化学习，引导大模型不再“逢题必深思熟虑”，而是根据问题难度自主决定“是否思考”“思考多少”。在多个数学

## 校正 (Correcting)

校正检索（Corrective-RAG, C-RAG）是引入一个“自我反思”或“自我修正”的循环，在生成答案之前，对检索到的文档质量进行评估，并根据评估结果采取不同的行动。

![](images/4_5_4.webp)

C-RAG 的工作流程可以概括为 “检索-评估-行动” 三个阶段：

1. 检索 (Retrieve) ：与标准 RAG 一样，首先根据用户查询从知识库中检索一组文档。

2. 评估 (Evaluate) ：一个“检索评估器 (Retrieval Evaluator)”会判断每个文档与查询的相关性，并给出“正确 (Correct)”、“不正确 (Incorrect)”或“模糊 (Ambiguous)”的标签。

3. 行动 (Act) ：根据评估结果，系统会进入不同的知识修正与获取流程：

    - 果评估为“正确”：系统会进入“知识精炼 (Knowledge Refinement)”环节。将原始文档分解成更小的知识片段 (strips)，过滤掉无关部分，然后重新组合成更精准、更聚焦的上下文，再送给大模型生成答案。

    - 如果评估为“不正确”：系统认为内部知识库无法回答问题，触发“知识搜索 (Knowledge Searching)”。对原始查询进行“查询重写 (Query Rewriting)”，生成一个更适合搜索引擎的查询，然后进行 Web 搜索，用外部信息来回答问题。

    - 如果评估为“模糊”：同样会触发“知识搜索”，但通常会直接使用原始查询进行 Web 搜索，以获取额外信息来辅助生成答案。

C-RAG 通过增加了一个“事实核查”层，能够在检索失败时主动寻求外部帮助，从而有效减少幻觉，提升答案的准确性和可靠性。